# Wordle Model Training Notebook

This notebook demonstrates how to train your ML model on Google Colab using a powerful GPU (like an A100) and how to save the trained model to Google Drive. 

You can later deploy this trained model in your web application. Modify the cells below to suit your specific training code and dataset.

In [ ]:
# Mount Google Drive to save model checkpoints and data
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Install any additional dependencies if needed (uncomment the following lines):
# !pip install torch torchvision torchaudio
# !pip install fastapi uvicorn

# Note: PyTorch is usually pre-installed in Colab.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.cuda import amp
from torch.utils.data import Dataset, DataLoader
import os
import time

print('Using device:', torch.device('cuda' if torch.cuda.is_available() else 'cpu'))

In [ ]:
class HeuristicScoringModel(nn.Module):
    def __init__(self, input_size, hidden_size=1024, num_hidden_layers=3, dropout=0.3):
        super(HeuristicScoringModel, self).__init__()
        layers = []
        layers.append(nn.Linear(input_size, hidden_size))
        layers.append(nn.ReLU())
        layers.append(nn.Dropout(dropout))
        for _ in range(num_hidden_layers - 1):
            layers.append(nn.Linear(hidden_size, hidden_size))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout))
        layers.append(nn.Linear(hidden_size, 1))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

print('HeuristicScoringModel defined.')

In [ ]:
class ExampleDataset(Dataset):
    """A simple dataset for demonstration purposes."""
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]


def train_model(model, dataloader, criterion, optimizer, device, scaler, num_epochs=5, scheduler=None):
    model.train()
    for epoch in range(num_epochs):
        epoch_loss = 0.0
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            with amp.autocast():
                outputs = model(inputs)
                loss = criterion(outputs, labels)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            if scheduler is not None:
                scheduler.step()

            epoch_loss += loss.item() * inputs.size(0)
        epoch_loss /= len(dataloader.dataset)
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")


def evaluate_model(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0.0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            with amp.autocast():
                outputs = model(inputs)
                loss = criterion(outputs, labels)
            total_loss += loss.item() * inputs.size(0)
    total_loss /= len(dataloader.dataset)
    print(f"Evaluation Loss: {total_loss:.4f}")
    return total_loss

print('Dataset and training functions defined.')

In [ ]:
# For demonstration, create a dummy dataset
input_size = 100  # Update this if using a different input dimension
num_samples = 1000

import numpy as np

# Create random data and labels
train_data = [torch.randn(input_size) for _ in range(num_samples)]
train_labels = [torch.randn(1) for _ in range(num_samples)]

# Split into training and validation sets
split = int(0.8 * num_samples)
train_dataset = ExampleDataset(train_data[:split], train_labels[:split])
val_dataset = ExampleDataset(train_data[split:], train_labels[split:])

batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

print(f"Training samples: {len(train_dataset)}, Validation samples: {len(val_dataset)}")

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = HeuristicScoringModel(input_size).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
scaler = amp.GradScaler()

# Optional: Define a learning rate scheduler
# scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
scheduler = None

num_epochs = 5  # Increase the number of epochs as needed
start_time = time.time()
train_model(model, train_loader, criterion, optimizer, device, scaler, num_epochs=num_epochs, scheduler=scheduler)
print(f"Training completed in {time.time() - start_time:.2f} seconds")

# Evaluate on validation set
evaluate_model(model, val_loader, criterion, device)

In [ ]:
# Save the trained model to Google Drive
model_save_path = '/content/drive/MyDrive/wordle_model.pth'
torch.save(model.state_dict(), model_save_path)
print(f'Model saved to {model_save_path}')